# __Exploratory Data Analysis (EDA)__
_Projet Analyse de sentiments — Exploration initiale des tweets_


In [1]:
import pandas as pd

# Charger le fichier parquet optimisé
df = pd.read_parquet("../data/processed/tweets.parquet")

# Aperçu des données
df.head()


,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## __Dimensions du dataset__


In [2]:
df.shape


(1600000, 2)

## __Répartition des labels__


In [3]:
df['label'].value_counts(normalize=True)


label
0    0.5
4    0.5
Name: proportion, dtype: float64

## __Statistiques sur la longueur des tweets__


In [4]:
df['len'] = df['text'].str.len()
df['len'].describe()


count    1.600000e+06
mean     7.409011e+01
std      3.644114e+01
min      6.000000e+00
25%      4.400000e+01
50%      6.900000e+01
75%      1.040000e+02
max      3.740000e+02
Name: len, dtype: float64